In [10]:
from qiskit.opflow import  I, X, Y, Z
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2

## Hamiltonian

We load the Hamiltonian from an external file. For this demonstration, the Hamiltonian was taken directly from [Kandala et al.](https://www.nature.com/articles/nature23879).

In [12]:
from qiskit.opflow import I, X, Y, Z, PauliSumOp

# Define the coefficients and Pauli words

lines = None

with open('BeH2_ham.txt') as f:
    lines = [line.rstrip() for line in f]

paulis = []
coeffs = []

for i in range(0, len(lines), 2):
    paulis.append(lines[i])
    coeffs.append(float(lines[i+1]))

# -----------------
    
ops = []

for coeff, pauli in zip(coeffs, paulis):
    if pauli[0] == 'I':
        op = I
    elif pauli[0] == 'X':
        op = X
    elif pauli[0] == 'Y':
        op = Y
    elif pauli[0] == 'Z':
        op = Z

    for p in pauli[1:]:
        if p == 'I':
            op = op ^ I
        elif p == 'X':
            op = op ^ X
        elif p == 'Y':
            op = op ^ Y
        elif p == 'Z':
            op = op ^ Z
    
    op = coeff * op
    ops.append(op)

hh = sum(ops)


## Ansatz
We construct a similar Ansatz as [Kandala et al.](https://www.nature.com/articles/nature23879).

In [14]:
hamiltonian_op = hh

In [30]:
from qiskit.circuit import QuantumCircuit, ParameterVector

# Define the Ansatz
wavefunction = QuantumCircuit(7)
params = ParameterVector('theta', length=24)
it = iter(params)

for _ in range(6):
    wavefunction.rx(next(it), _)
    wavefunction.rz(next(it), _)

wavefunction.cx(0, 1)
wavefunction.cx(1, 3)
wavefunction.cx(3, 5)
wavefunction.cx(5, 6)
wavefunction.cx(5, 4)

for _ in range(6):
    wavefunction.rx(next(it), _)
    wavefunction.rz(next(it), _)

## VQE
This is an example of a VQE code in Qiskit. The following code will not give optimal results for the moment, as the Ansatz and the VQE's parameters need to be better chosen. This is here for demonstration purposes only.

In [31]:
from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian
grad = Gradient(grad_method='lin_comb')

In [38]:
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
optimizer = SPSA(maxiter=100)

In [39]:
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance

backend = Aer.get_backend('aer_simulator')
qi = QuantumInstance(backend)

In [40]:
from qiskit.algorithms import VQE

In [41]:
vqe = VQE(var_form, optimizer=optimizer, quantum_instance=qi)

In [42]:
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian_op)
print(result)
optimizer_evals = result.optimizer_evals

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 200,
    'eigenstate': {   '000001': 0.03125,
                      '000010': 0.08267972847076846,
                      '000011': 0.04419417382415922,
                      '000100': 0.03125,
                      '000110': 0.05412658773652741,
                      '001000': 0.04419417382415922,
                      '001001': 0.03125,
                      '001010': 0.14320549046737,
                      '001011': 0.05412658773652741,
                      '001100': 0.06987712429686843,
                      '001101': 0.05412658773652741,
                      '001110': 0.08267972847076846,
                      '001111': 0.05412658773652741,
                      '010000': 0.0625,
                      '010001': 0.05412658773652741,
                      '010010': 0.7146622891268295,
                      '010011': 0.3603925810834624,
                      '010100': 0.2072890493972125,
                      '010101':